In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from beartype import beartype as typed
from beartype.door import die_if_unbearable as assert_type
from datasets import load_dataset
from jaxtyping import Float, Int, Bool
from typing import Callable
from torch import Tensor as TT
from transformers import AutoModelForCausalLM, AutoTokenizer
from einops import einops as ein

%load_ext autoreload
%autoreload 2

In [ ]:
seed_everything(42)


n_samples = 10**4
dict_dim = 100

gt: Int[TT, "n_samples hidden_dim"] = t.bernoulli(t.full((n_samples, hidden_dim), 0.1))
obs: Float[TT, "n_samples input_dim"] = gt @ emb


# sae = SparseAutoEncoder(input_dim, dict_dim)
dist = t.zeros((hidden_dim, dict_dim))

for it in range(10):
    fit_sae(sae, obs, hidden_dim, lr=0.01, l1=0.2, epochs=20, noise_std=0.0)
    guess = sae.encode(obs)

    for true_feature in range(hidden_dim):
        for dict_feature in range(dict_dim):
            y = gt[:, true_feature].float()
            p = guess[:, dict_feature].detach()
            dist[true_feature, dict_feature] = F.cosine_similarity(p, y, dim=-1)

    plt.figure(figsize=(4, 4))
    plt.imshow(dist)
    plt.show()
    best = dist.max(dim=1).values
    print(ls(best.mean()), ls(best))

In [ ]:
from utils import ls, sh
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


for i in range(30):
    if gt[i].sum() == 0:
        continue
    print("mse", F.mse_loss(obs[i], sae(obs[i])[0]).mul(32).round().div(32).item())
    print(ls(gt[i]))
    print(ls(guess[i]))
    print(ls(y))
    print()